##### ver7　Normalization、onehotencordeinなどにNNを追加



##### ver8 上記にEarlyStopping追加



# ver5 epoch 200で提出

下記を参照
https://www.kaggle.com/takeshikobayashi/try-to-use-nn-baseline-78a3ae/edit/run/53017979

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# データロード

In [ ]:
# coding: utf-8

import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import datatable as dt

#  初始化一个随机数种子
SEED = 1111
np.random.seed(SEED)
tf.random.set_seed(SEED)

print('Loading data...')

train_datatable = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')  # 用datatable会快一点
train = train_datatable.to_pandas()
del train_datatable

display(train)


#前処理
print('Data preprocessing...')

train = train.query('date > 85').reset_index(drop = True)   # 只保留第86天及以后的data
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)  # 注意，这种暴力mean的方案会数据穿越导致overfitting，所以应该滚动mean会更加“正确”，至于效果是不是一定好，可能是个玄学

train_date = train['date'].values

# features = []
# for item in train.columns:
#     if 'feature' in item:
#         features.append(item)

# f_mean = np.mean(train[features[1:]].values,axis=0)  # 算一下每个feature的均值，用于缺失值填充

print('Done!')

In [ ]:
#train['feature_test'] = train['feature_41'] + train['feature_42'] + train['feature_43']
#train['feature_test']


#import matplotlib.pyplot as plt

#fig, ax = plt.subplots(figsize=(15, 5))
#balance= pd.Series(train['feature_test']).cumsum()
#ax.set_xlabel ("Trade", fontsize=18)
#ax.set_ylabel ("Cumulative resp", fontsize=18);
#balance.plot(lw=3);

#train['feature_test'].describe()

In [ ]:
# 正規化
print('Normalization...')

features_mean = train.loc[:, train.columns.str.contains('feature')].mean()
features_std  = train.loc[:, train.columns.str.contains('feature')].std()
train_features_normorlization = (train.loc[:, train.columns.str.contains('feature')] - features_mean) / features_std

print('【train_features_normorlization】【features_mean】【features_std】Done!')

train_features_normorlization.head()

# 特徴量追加


In [ ]:
from sklearn.preprocessing import OneHotEncoder
print('one-hot...')

#feature_0をone_hot_encorder

f0_onehot_encoder = OneHotEncoder()
print('fitting...', end='')
f0_onehot_encoder.fit(train[['feature_0']])

print('Done!\ntransforming...', end='')
f0_onehot = f0_onehot_encoder.transform(train[['feature_0']]).toarray()
##DataFrameの場合
#f0_onehot = f0_onehot_encoder.transform(train[['feature_0']])
print('Done!')
print(f0_onehot)

# ----------------------------- #
#     f41_f42_f43 = stock_id    #
#     
# ----------------------------- #
# 
# train['stock_id'] = train['feature_41'] + train['feature_42'] + train['feature_43']
# 
# train['stock_counts_bin'] = pd.DataFrame({
#     'stock_counts_bin':pd.qcut(train['stock_id'].rank(method='first'), 20, labels=False)  # 等频20桶
# })

# stock_id_onehot_encoder = OneHotEncoder(handle_unknown='ignore')
# print('fitting...', end='')
# stock_id_onehot_encoder.fit(train[['stock_counts_bin']])
# print('Done!\ntransforming...')
# stock_id_onehot = stock_id_onehot_encoder.transform(train[['stock_counts_bin']]).toarray()

# print('【f0_onehot】【stock_id_onehot】Done!')



# 看一下各个stock_id出现的次数
train['stock_id'] = train['feature_41'] + train['feature_42'] + train['feature_43']
# 对该股票历史出现的频次做一个分桶
stock_counts_bin = pd.DataFrame({
    'stock_id':train['stock_id'].values,
    'stock_counts_bin':pd.qcut(train['stock_id'].rank(method='first'), 20, labels=False)  # 等频20桶
})

stock_id_onehot_encoder = OneHotEncoder(handle_unknown='ignore')
print('fitting...', end='')
stock_id_onehot_encoder.fit(stock_counts_bin[['stock_counts_bin']])
print('Done!\ntransforming...')
stock_id_onehot = stock_id_onehot_encoder.transform(stock_counts_bin[['stock_counts_bin']]).toarray()
#stock_id_onehot = stock_id_onehot_encoder.transform(stock_counts_bin[['stock_counts_bin']])

print(stock_id_onehot[-1])
print(stock_id_onehot.shape)
display(stock_counts_bin)

print('【f0_onehot】【stock_id_onehot】Done!')

#stock_id_onehot = pd.DataFrame(stock_id_onehot)

In [ ]:
stock_id_onehot.shape

# マージ

In [ ]:
print('Merge and drop features...')

#train_feature = train.loc[:, train.columns.str.contains('feature')].drop(['feature_0','feature_41','feature_42','feature_43'], axis=1)
train_nparray = np.delete(train_features_normorlization.values, [0,41,42,43], axis=1)

print(train_nparray.shape)
print(f0_onehot.shape)
print(stock_id_onehot.shape)
#print(np.concatenate([train_nparray, f0_onehot, stock_id_onehot],axis=1).shape)
#print(train_nparray.join(stock_id_onehot).shape)

print('【train_nparray】Done!')

#f0_onehot = pd.DataFrame(f0_onehot.toarray())

#name = ["f0_onehot_0","f0_onehot_1"]


In [ ]:
train_nparray.shape

# 特徴量と目的変数設定

In [ ]:
print('Make X and y...')


X_train = np.concatenate([train_nparray, f0_onehot, stock_id_onehot],axis=1)

#X_train = np.concatenate([f0_onehot, stock_id_onehot],axis=1)

#X_train = train_feature.join(f0_onehot)
#X_train = X_train.join(stock_id_onehot)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
#y_train = pd.DataFrame(y_train)

# print(X_train[0])
del train_nparray, f0_onehot, stock_id_onehot, train_features_normorlization

print('【X_train】【y_train】Done!')

In [ ]:
X_train.shape


# モデル設定（過去のものを転用）

In [ ]:

### https://www.kaggle.com/takeshikobayashi/iiml-neural-network-starter 


def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)    
        
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('sigmoid')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model

https://www.kaggle.com/takeshikobayashi/try-to-use-nn-baseline-78a3ae 参照

# 別モデル

In [ ]:
'''
print('Create a easy NN model...')

HIDDEN_LAYER = [150, 150, 150]
TARGET_NUM = 5   # 优化那5个resp

# np.set_printoptions(threshold=1000)
# print(X_train[0])

input = tf.keras.layers.Input(shape=(X_train.shape[1], ))

x = tf.keras.layers.BatchNormalization()(input)
x = tf.keras.layers.Dropout(0.2)(x)
for units in HIDDEN_LAYER:
    x = tf.keras.layers.Dense(units)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)  # 除了ReLU还可以试试别的，后面可以做一个多模型融合
    x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(TARGET_NUM)(x)

output = tf.keras.layers.Activation("sigmoid")(x)

model = tf.keras.models.Model(inputs=input, outputs=output)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics   = tf.keras.metrics.AUC(name="AUC"),
    loss      = tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-2),
)

print('Done!')
'''

In [ ]:
'''
print('Train NN...')

history = model.fit(X_train, y_train, epochs=200, batch_size=5000)
models = []
models.append(model)

print('Done!')
'''

In [ ]:
'''
from sklearn.model_selection import GroupKFold
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import gc
##ver1
batch_size = 4096
hidden_units = [384, 896, 896, 394]
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3

model = create_mlp(X_train.shape[1], y_train.shape[1], hidden_units, dropout_rates, label_smoothing, learning_rate)

rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, 
                            min_delta = 1e-4, mode = 'max')
#ckp = ModelCheckpoint(ckp_path, monitor = 'val_AUC', verbose = 0, 
#                          save_best_only = True, save_weights_only = True, mode = 'max')
es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
                       baseline = None, restore_best_weights = True, verbose = 0)

model.fit(X_train, y_train, epochs = 1000)
'''

In [ ]:
from sklearn.model_selection import GroupKFold
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import gc

##ver1
batch_size = 4096
hidden_units = [384, 896, 896, 394]
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3

#oof = np.zeros(len(train['action']))
oof = np.zeros(len(y_train))

gkf = GroupKFold(n_splits = 5)

#for fold, (tr, te) in enumerate(gkf.split(X_train, y_train, train['date'].values)):
for fold, (tr, te) in enumerate(gkf.split(X_train, y_train, train_date)):
    
    #X_tr, X_val = train.loc[tr, features].values, train.loc[te, features].values
    #y_tr, y_val = train.loc[tr, 'action'].values, train.loc[te, 'action'].values
    
    #X_tr = X_train.iloc[tr, : ].values
    #X_val = X_train.iloc[te, : ].values
    
    #y_tr = y_train.iloc[tr, :].values
    #y_val = y_train.iloc[te, :].values
    
    X_tr = X_train[tr, : ]
    X_val = X_train[te, : ]
    
    y_tr = y_train[tr, :]
    y_val = y_train[te, :]
    
    
    ckp_path = f'JSModel_{fold}.hdf5'
    
    model = create_mlp(X_tr.shape[1], y_tr.shape[1], hidden_units, dropout_rates, label_smoothing, learning_rate)
    
    rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, 
                            min_delta = 1e-4, mode = 'max')
    ckp = ModelCheckpoint(ckp_path, monitor = 'val_AUC', verbose = 0, 
                          save_best_only = True, save_weights_only = True, mode = 'max')
    es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
                       baseline = None, restore_best_weights = True, verbose = 0)
    
    model.fit(X_tr, y_tr, validation_data = (X_val, y_val), epochs = 1000, 
              batch_size = batch_size, callbacks = [rlr, ckp, es], verbose = 0)
    #model.fit(X_tr, y_tr)
              
    oof[te] += model.predict(X_val, batch_size = batch_size * 4).ravel()
    score = roc_auc_score(y_val, oof[te])
    print(f'Fold {fold} ROC AUC:\t', score)
    
    # Finetune 3 epochs on validation set with small learning rate
    model = create_mlp(X_tr.shape[1], y_tr.shape[1], hidden_units, dropout_rates, label_smoothing, learning_rate)
    model.load_weights(ckp_path)
    model.fit(X_val, y_val, epochs = 3, batch_size = batch_size, verbose = 0)
    model.save_weights(ckp_path)
   
    K.clear_session()
    del model
    rubbish = gc.collect()

In [ ]:
X_tr.shape

In [ ]:
X_val.shape

In [ ]:
y_tr.shape

# 実験

In [ ]:

from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


from sklearn.model_selection import train_test_split
 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train,
                                                        y_train, 
                                                        test_size = 0.2,
                                                        train_size = 0.8,#教師データ少なくなるのが怖いので4:1で分割
                                                        stratify = y_train)


batch_size = 4096
hidden_units = [384, 896, 896, 394]
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3


#モデルの構造
model2 = Sequential()
model2.add(tf.keras.layers.Input(shape = (X_train2.shape[1], )))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(150))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Activation(tf.keras.activations.swish))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(150))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Activation(tf.keras.activations.swish))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(5))
model2.add(tf.keras.layers.Activation('sigmoid'))


model2.compile(loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing),
              optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
              metrics = tf.keras.metrics.AUC(name = 'AUC'))

In [ ]:

hist = model2.fit(X_train2, y_train2)

In [ ]:
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

In [ ]:
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

batch_size = 4096
hidden_units = [384, 896, 896, 394]
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3



#rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, 
#                       min_delta = 1e-4, mode = 'max')
#ckp = ModelCheckpoint(ckp_path, monitor = 'val_AUC', verbose = 0, 
#                      save_best_only = True, save_weights_only = True, mode = 'max')
#es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
#                   baseline = None, restore_best_weights = True, verbose = 0)


In [ ]:
test = y_test2[4]

In [ ]:
traintest = y_train2[4]

In [ ]:
#model = create_mlp(X_train2.shape[1], 1, hidden_units, dropout_rates, label_smoothing, learning_rate)
model = create_mlp(X_train2.shape[1], y_train2.shape[1], hidden_units, dropout_rates, label_smoothing, learning_rate)

In [ ]:
model.fit(X_train2, y_train2, validation_data = (X_test2, y_test2), epochs = 200, batch_size = batch_size, verbose = 0)

#model.fit(X_test2, test)

In [ ]:
y_tr

In [ ]:
import gc

gc.collect()

In [ ]:
#x = tf.keras.layers.Dense(TARGET_NUM)(x)
#
#output = tf.keras.layers.Activation("sigmoid")(x)
#
#model = tf.keras.models.Model(inputs=input, outputs=output)
#model.compile(
#    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
#    metrics   = tf.keras.metrics.AUC(name="AUC"),
#    loss      = tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-2),
#)

In [ ]:
X_train.shape

In [ ]:

def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)    
        
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)
    #out = tf.keras.layers.Activation("softmax")(x)
    
    
    
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model


In [ ]:
"""
def create_mlp2(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):

    input = tf.keras.layers.Input(shape=(num_columns, ))
    
    x = tf.keras.layers.BatchNormalization()(input)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    
    for units in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)
        
    x = tf.keras.layers.Dense(num_labels)(x)
    
    output = tf.keras.layers.Activation("sigmoid")(x)
    
    return model
"""

In [ ]:
##ver1
batch_size = 4096
hidden_units = [384, 896, 896, 394]
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3

In [ ]:
Train = True

#ver8 EarlyStopping追加

if Train:
    
    from keras.callbacks import EarlyStopping
    
#early_stopping =  EarlyStopping(
#                            monitor='loss',
#                            min_delta=0.000,
#                            patience=10,
#)
    
    early_stopping = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
                           baseline = None, restore_best_weights = True, verbose = 0)
    
    print('Train NN...')
    
    model = create_mlp(X_train.shape[1], y_train.shape[1], hidden_units, dropout_rates, label_smoothing, learning_rate)
    
    
    history = model.fit(X_train, y_train, epochs=200, batch_size=batch_size,callbacks=[early_stopping]) # CallBacksに設定
    models = []
    models.append(model)
    
    print('Done!')
    
else:
    
    model = create_mlp(X_train.shape[1], y_train.shape[1], hidden_units, dropout_rates, label_smoothing, learning_rate)
    model.load_weights('../input/iimlnnmodel-part2/NNmodel_part2.hdf5')
    
    
    models = []
    models.append(model)
    
    print('weight load Done!')
    
#    for i in range(num_models):
#        clf = create_mlp(len(features), 1, hidden_units, dropout_rates, label_smoothing, learning_rate)
#        clf.load_weights(f'./JSModel_{i}.hdf5')
#     clf.load_weights(f'./JSModel_{i}.hdf5')
#        models.append(clf)   
    

In [ ]:
#重みだけ保存
model.save_weights('NNmodel_part2.hdf5')

In [ ]:
"""
from sklearn.model_selection import GroupKFold

oof = np.zeros(len(y_train))
gkf = GroupKFold(n_splits = 5)
for fold, (tr, te) in enumerate(gkf.split(y_train, y_train, X_train['date'].values)):
    
    X_tr, X_val = X_train.loc[tr, features].values, X_train.loc[te, features].values
    y_tr, y_val = X_train.loc[tr, 'action'].values, X_train.loc[te, 'action'].values
  
    ckp_path = f'JSModel_{fold}.hdf5'
    
    model = create_mlp(X_tr.shape[1], 1, hidden_units, dropout_rates, label_smoothing, learning_rate)
    
    rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, 
                            min_delta = 1e-4, mode = 'max')
    ckp = ModelCheckpoint(ckp_path, monitor = 'val_AUC', verbose = 0, 
                          save_best_only = True, save_weights_only = True, mode = 'max')
    es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
                       baseline = None, restore_best_weights = True, verbose = 0)
    model.fit(X_tr, y_tr, validation_data = (X_val, y_val), epochs = 1000, 
              batch_size = batch_size, callbacks = [rlr, ckp, es], verbose = 0)
              
    oof[te] += model.predict(X_val, batch_size = batch_size * 4).ravel()
    score = roc_auc_score(y_val, oof[te])
    print(f'Fold {fold} ROC AUC:\t', score)
  
    # Finetune 3 epochs on validation set with small learning rate
    model = create_mlp(X_tr.shape[1], 1, hidden_units, dropout_rates, label_smoothing, learning_rate / 100)
    model.load_weights(ckp_path)
    model.fit(X_val, y_val, epochs = 3, batch_size = batch_size, verbose = 0)
    model.save_weights(ckp_path)
   
    K.clear_session()
    del model
    rubbish = gc.collect()

"""

# 久野ツール関数定義

In [ ]:
"""
env_called = False
iter_test = None

def calc_score(train_df, pred_df):
    train_test_result = train_df[["date", "weight", "resp"]].copy().reset_index(drop=True)
    train_test_result["resp_weight_action"] = pred_df["action"] * train_test_result["weight"] * train_test_result["resp"] 
    px = train_test_result.groupby("date")["resp_weight_action"].sum()
    num_of_i = len(px)
    t = sum(px)/np.sqrt(sum(px ** 2)) * np.sqrt(250/num_of_i)
    u = min(max(t, 0), 6) * sum(px)
    return u

def pred_action(test_df, models, f=np.median, th=0.5):
        ##特徴量追加分　追記
            
        add_feature(test_df)
            
        #test_df['feature_cross_41_42_43'] = test_df['feature_42'] +  test_df['feature_43'] +  test_df['feature_44']
        #test_df['feature_cross_1_2'] = test_df['feature_1'] / (test_df['feature_2'] + 1e-5)
        
        ##以上
        if test_df['weight'].item() <=0:
            return 0
        else:
            
            
            
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt[:, 1:].sum()):
                #print(f_mean)
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
            pred = f(pred)
            return np.where(pred >= th, 1, 0).astype(int)

def pred_action_burst(test_df, models, f=np.median, th=0.5):
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            #print(f_mean)
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred = np.where(pred >= th, 1, 0).astype(int)
        fil = (test_df['weight'] == 0)
        pred[fil] = 0
        return pred

def submit(models, example_test_df=None, burst=False):
    if example_test_df is None:
        print("submit mode")
        global iter_test
        global env_called
        if not env_called:
            import janestreet
            env = janestreet.make_env()
            iter_test = env.iter_test()
            env_called = True
    else:
        print("example test mode")
        # 事前準備
        print("preparing dataset...")
        if not burst:
            test_iter = [pd.DataFrame([s[1]]) for s in example_test_df.iterrows()] 
            pred_iter = [pd.DataFrame([0], columns=["action"]) for _ in range(len(example_test_df))]
            iter_test = zip(test_iter, pred_iter)
        else:
            print("bust mode (it's fast but it doesn't use time-serires module. Check pred_action_burst carefully.)")
            batch_size = 4096
            row_len = len(example_test_df)
            test_iter = [example_test_df.iloc[i * batch_size: (i+1) * batch_size ] for i in range(row_len//batch_size + 1)] 
            pred_iter = [pd.DataFrame([0] * batch_size, columns=["action"]) for i in range(row_len//batch_size + 1)]
            pred_iter[-1] = pd.DataFrame([0] * (row_len%batch_size), columns=["action"])

            iter_test = zip(test_iter, pred_iter)
        print("preparing dataset... Done")

    f = lambda x: np.median(x, axis=1)
    th = 0.5
    models = models[-3:]
    
    print("predicting...")
    if example_test_df is None:
        for (test_df, pred_df) in tqdm(iter_test):
            
            ##特徴量追加分　追記
            
            add_feature(test_df)
            
            ##以上
            
            pred_df.action = pred_action(test_df, models, f, th)
            env.predict(pred_df)
        return pd.DataFrame()
    elif not burst:
        submit_ex_result = []
        for (test_df, pred_df) in tqdm(iter_test):
            
            ##特徴量追加分　追記
            
            add_feature(test_df)
            
            ##以上
            
            pred_df.action = pred_action(test_df, models, f, th)
            submit_ex_result.append(pred_df)  # env.predict(pred_df)
        return pd.concat(submit_ex_result).reset_index(drop=True)
    else:
        submit_ex_result = []
        for (test_df, pred_df) in tqdm(iter_test):
            
            ##特徴量追加分　追記
            add_feature(test_df)
            
            ##以上
            
            pred_df.action = pred_action_burst(test_df, models, f, th)
            submit_ex_result.append(pred_df)  # env.predict(pred_df)
        return pd.concat(submit_ex_result).reset_index(drop=True)  
    
    
def add_feature(test_df):
    
        #test_sample = test_df.loc[:, test_df.columns.str.contains('feature')]
        test_sample = test_df
        
        #feature_list = test_sample.columns
        #features
        
        #test_sample = test_sample.values
        
        # 如果这行样本里面有NaN的话（忽略离散特征feature_0）用train数据集里的mean替换NaN
        if np.isnan(test_sample.sum()):         
            test_sample = np.nan_to_num(test_sample) + np.isnan(test_sample) * features_mean.values
        
        # Normalization
        test_sample = (test_sample - features_mean.values) / features_std.values
        
        #NumpyからPandasに戻す
        test_sample = pd.DataFrame(test_sample,columns = features)
        
        
        # one-hot
        test_f0_onehot = f0_onehot_encoder.transform(test_df[['feature_0']]).toarray()
        
        #stockid_df = test_sample['feature_41'] + test_sample['feature_42'] + test_sample['feature_43']
        
        test_sample['stock_id'] = test_sample['feature_41'] + test_sample['feature_42'] + test_sample['feature_43']
        #test_sample_stock_id = test_sample[0,41] + test_sample[0,42] + test_sample[0,43]
        
        
        #stock_counts_bin = pd.DataFrame({
        #    'stock_id':test_sample['stock_id'].values,
        #    'stock_counts_bin':pd.qcut(test_sample['stock_id'].rank(method='first'), 20, labels=False)  # 等频20桶
        #    })
        
        
        test_stock_id_onehot = stock_id_onehot_encoder.transform(test_sample[['stock_id']]).toarray()
        #test_stock_id_onehot = stock_id_onehot_encoder.transform(test_sample[['stock_counts_bin']]).toarray()
        stock_counts_bin
        
        test_sample = test_sample.drop(['stock_id'],axis = 1)
        
        # drop and merge
        test_sample = np.delete(test_sample.values, [0,41,42,43], axis=1)
        test_sample = np.concatenate([test_sample, test_f0_onehot, test_stock_id_onehot],axis=1)
        
        return test_df
        
"""

In [ ]:
"""
%%time

#feature入れなおし
#features = [c for c in X_train.columns if 'feature' in c]

#fmean 入れなおし
#f_mean = X_train[features[1:]].mean().values

f_mean = features_mean

"""

# predict

In [ ]:
## 変更点
## numpyばかりでよくわからんからpandasに変更する。

THRESHOLD = 0.5   # 试了一个对照组，还是0.5比较好用

import janestreet
from tqdm import tqdm
janestreet.make_env.__called__ = False
env = janestreet.make_env()

print(features_mean.values)

i = 1

for (test_df, pred_df) in tqdm(env.iter_test()):
    
    
    if test_df['weight'].item() > 0:
        
        #test_sample = test_df.loc[:, test_df.columns.str.contains('feature')].values  
        test_sample = test_df.loc[:, test_df.columns.str.contains('feature')]
        
        feature_list = test_sample.columns
        
        test_sample = test_sample.values
        
        # 如果这行样本里面有NaN的话（忽略离散特征feature_0）用train数据集里的mean替换NaN
        if np.isnan(test_sample.sum()):         
            test_sample = np.nan_to_num(test_sample) + np.isnan(test_sample) * features_mean.values
        
        # Normalization
        test_sample = (test_sample - features_mean.values) / features_std.values
        
        #NumpyからPandasに戻す
        test_sample = pd.DataFrame(test_sample,columns = feature_list)
        
        
        # one-hot
        test_f0_onehot = f0_onehot_encoder.transform(test_df[['feature_0']]).toarray()
        
        #stockid_df = test_sample['feature_41'] + test_sample['feature_42'] + test_sample['feature_43']
        
        test_sample['stock_id'] = test_sample['feature_41'] + test_sample['feature_42'] + test_sample['feature_43']
        #test_sample_stock_id = test_sample[0,41] + test_sample[0,42] + test_sample[0,43]
        
        
        #stock_counts_bin = pd.DataFrame({
        #    'stock_id':test_sample['stock_id'].values,
        #    'stock_counts_bin':pd.qcut(test_sample['stock_id'].rank(method='first'), 20, labels=False)  # 等频20桶
        #    })
        
        
        test_stock_id_onehot = stock_id_onehot_encoder.transform(test_sample[['stock_id']]).toarray()
        #test_stock_id_onehot = stock_id_onehot_encoder.transform(test_sample[['stock_counts_bin']]).toarray()
        stock_counts_bin
        
        test_sample = test_sample.drop(['stock_id'],axis = 1)
        
        # drop and merge
        test_sample = np.delete(test_sample.values, [0,41,42,43], axis=1)
        test_sample = np.concatenate([test_sample, test_f0_onehot, test_stock_id_onehot],axis=1)
        
        #確認用
        if i == 1:
        
            check = test_sample
        
        else:
            check = np.append(check,test_sample, axis=0)
        
        #予測
        pred = model(test_sample, training = False).numpy()
        
        # version = 1
        pred = np.median(pred)
        
        i = i + 1
        
        # version = 2
#         pred = np.mean(pred)
        
        # version = 3
#         pred = pred[3]
        
        pred_df.action = np.where(pred >= THRESHOLD, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

In [ ]:
check

In [ ]:
pred2 = model(check, training = False).numpy()

In [ ]:
pred2